Let's start with imports:

In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
from astropy.table import Table
from astropy.io import fits
from astropy.utils.data import download_file
from astropy.visualization import PercentileInterval, AsinhStretch
from photutils.aperture import aperture_photometry, CircularAperture, CircularAnnulus, ApertureStats
import matplotlib
matplotlib.use('TkAgg')
from numpy import *
from pylab import *

Now we'll define some universal constants, to be changed as seen fit:

In [ ]:
sizestring = '240'
size = 240
radius = 15
cone = 300/3600 #in degrees
format = 'fits'
position = [size/2, size/2]

Here are some pared-down functions from the PanStarrs tutorial code to create a url and download an image:

In [ ]:
def geturl(ra, dec, size, filter):

      '''
      Function that creates a PanStarrs FITS url to download.

      Parameters:
      -----------
      ra  : float
            J2000 RA of center of image in degrees

      dec : float
            J2000 Dec of center of image in degrees

      size : integer
            image size in pixels where 240 pixels is 60 arcsec

      filter : string
            desired filter for the image (g, r, i, z, or y)

      Returns:
      --------
      url : string
            desired url to unpack for a FITS file
      '''

      format, output_size, color = "fits", None, False


      service = "https://ps1images.stsci.edu/cgi-bin/ps1filenames.py"

      url = ("{service}?ra={ra}&dec={dec}&size={size}&format=fits"
      "&filters={filter}").format(**locals())

      table = Table.read(url, format='ascii')

      url = ("https://ps1images.stsci.edu/cgi-bin/fitscut.cgi?"
      "ra={ra}&dec={dec}&size={size}&format={format}").format(**locals())

      urlbase = url + "&red="
      url = []
      for filename in table['filename']:
            url.append(urlbase+filename)

      return url

This function runs the get_url() function and downloads the data.

In [ ]:
def readfile(ra, dec, size, filter):
      '''
      This function runs the get_url() function, creates a path, and downloads the FITS file data and header. 

      Parameters:
      -----------
      ra  : float
            J2000 RA of center of image in degrees

      dec : float
            J2000 Dec of center of image in degrees

            size : integer
            image size in pixels where 240 pixels is 60 arcsec

      filter : string
                  desired filter for the image (g, r, i, z, or y)

      Returns:
      -------
      data : numpy.ndarray
            data describing the FITS file

      header: astropy.io.fits.header.Header
              header for the FITS file containing information about the collection processes and image reduction
      '''
      
      url=geturl(ra, dec, size, filter)[0]
      path = download_file(url)
      hdu = fits.open(path)
      data = hdu[0].data
      header = hdu[0].header
      hdu.close()

      return [data, header]

This function runs the aperture photometry functions from photutils and calculates the error. 

The magnitude is found by the following equation:
$$
M = M_{ZPT} - 2.5\log_{10}{\frac{c \cdot g}{t_{exp}}} 
$$
...where $M_{ZPT}$ is the zero-point magnitude (25 for PanStarrs FITS files), $t_{exp}$ is the exposure time in seconds, $c$ is the counts per pixel, and $g$ is the photon-electron gain.

The error in this magnitude can be approximated by:
$$
\sigma_{M} \approx \frac{-2.5}{\ln{10} \cdot c} \sqrt{\left(A_{aperture} + \frac{A_{aperture}^2}{A_{BG}}\right)\sigma_{BG}^2}
$$

And the 3-sigma upper limit on non-detections is then found with:

$$
M_{3\sigma} = M_{ZPT} - 2.5\log{\left[\mu_{BG}+3\sigma_{BG}*\frac{g}{t_{exp}}\right]}\pm{\left|\frac{2.5}{\ln{10} \cdot c}\right|\sqrt{\sigma_{BG}^2\left(A_{aperture}+\frac{A_{aperture}^2}{A_{BG}}\right)}}
$$

In [ ]:
def getmagnitude(data, header, radius, position):

    '''
    Function that handles the aperture photometry and calculates associated error. 

    Parameters:
    ----------
    data : numpy.ndarray
           data describing the FITS file

    header : astropy.io.fits.header.Header
             header for the FITS file containing information about the collection processes and image reduction

    radius : integer
             radius of aperture in pixels

    position : list 
               contains x and y coordinates (in pixels) of the center of the aperture

    Returns:
    --------
    mag_and_error : list
                    contains either the magnitude and error or the 3-sigma upper limit and the error
                    if there is no detection, the error will be negative
    
    '''

    Texp = header['EXPTIME']
    ZPTmag = header['HIERARCH FPA.ZP']
    gain = header['HIERARCH CELL.GAIN']

    aperture = CircularAperture(position, radius)
    annulus = CircularAnnulus(position, radius+2, np.sqrt(2*(radius**2+2*radius+2))) 

    phot_table = aperture_photometry(data, aperture)
    ann_table = aperture_photometry(data, annulus)

    counts = phot_table['aperture_sum'] - (ann_table['aperture_sum'])*(aperture.area/annulus.area)
    
    annstats  = ApertureStats(data, annulus)
    annvar = (annstats.std)**2

    count_error = np.sqrt(annvar*(aperture.area+(aperture.area**2)/annulus.area))
    error = (2.5/(np.log(10)*counts))*count_error
    upperlimcounts = annstats.mean+3*annstats.std
    Ne = counts*gain
    
    if Ne/Texp > 0:
        magnitude = ZPTmag - 2.5*np.log10(Ne/Texp)
        mag_and_error = [magnitude[0], error[0]] 
    else:
        Ne = ann_table['aperture_sum']*gain
        magnitude = ('3σ: '+str(ZPTmag - 2.5*np.log10(upperlimcounts*gain/Texp)))
        mag_and_error = [magnitude, str("{:.4f}".format(error[0]))]

    return mag_and_error

Here we'll define a function to plot the aperture and the annulus:

In [ ]:
def print_image(data):

    '''
    This function plots your aperture so you can see what is getting included. 

    Parameters:
    -----------
    data : numpy.ndarray
           data describing the FITS file

    Returns: a plot graphing the aperture and the annulus.
    '''

    data[np.isnan(data)] = 0.0
    # set contrast to something reasonable
    transform = AsinhStretch() + PercentileInterval(99.5)
    bfim = transform(data)

    theta = np.linspace( 0 , 2 * np.pi , 150 )
    a = np.cos( theta )
    b = np.sin( theta )

    figure, axes = plt.subplots( 1 )
    axes.plot(radius*a + size/2, radius*b + size/2)
    axes.plot((radius+2)*a + size/2, (radius+2)*b + size/2)
    axes.plot((np.sqrt(2*(radius**2+2*radius+2))*a) + size/2, (np.sqrt(2*(radius**2+2*radius+2))*b + size/2))
    axes.imshow(bfim,cmap="gray",origin="lower")
    axes.set_aspect( 1 )
    axes.set_title('PS1 i (fits)'+'inner radius = '+str(radius))
    axes.imshow(bfim,cmap="gray",origin="lower")
    axes.set_xlabel('RA (Deg)')
    axes.set_ylabel('Dec (Deg)')
    plt.show()

Put the FRBs you want to analyse here:

In [ ]:
'''
This should be a dictionary called FRBinfo with the entry format 'Name' : [RA, Dec]. For example:
FRBinfo = {
        'Arthur' : [274.334, 76.333],
        'Ford' : [221.333, 67.373],
        'Trillian' : [38.387, 54. 283],
        'Zaphod' : [353.373, 86.998]
}
'''

FRBinfo = {
        'Jessie': [286.575685, 8.8247238],
}

Now we'll run all of the above functions. This part will take about 15 seconds per FRB localization.

In [ ]:
idlist = ['RA', 'Dec', 'g', 'err_g', 'r', 'err_r', 'i', 'err_i', 'z', 'err_z', 'y', 'err_y']
df = pd.DataFrame(columns = idlist)
for name in FRBinfo:
    mag_list = []
    for filter in ['g', 'r', 'i', 'z', 'y']:
        file = readfile(FRBinfo[name][0], FRBinfo[name][1], size, filter)
        data = file[0]
        header = file[1]

        #let's plot the aperture in i-band, because i is a nice band.
        if filter == 'i':
            print_image(data)
        
        mag_and_error = getmagnitude(data, header, radius, position)
        mag_list.append(mag_and_error[0])
        mag_list.append(mag_and_error[1])

    line = [FRBinfo[name][0], FRBinfo[name][1]]
    line.extend(mag_list)
    line = pd.DataFrame([line], columns = idlist)
    df = pd.concat([df, line])

df = df.set_index(pd.Index(list(FRBinfo.keys())))
print(df)

Optionally, you can print this as a csv:

In [ ]:
file = open('Host Galaxies.csv', 'w')
file.write(df.to_csv())
file.close()